In [ ]:
#dependencies
import pandas as pd 
import gmaps
import requests
from config import g_key

In [ ]:
#store cities.csv in a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head(5)

In [ ]:
city_data_df.dtypes

In [ ]:
#configure gmaps to use my google api key
gmaps.configure(api_key=g_key)

In [ ]:
#general formula for heatmap
#assign locations to an array of lng and lat pairs
#assign weights varraible to some values
#assign the figure variable to gmaps.figure
#assign heatmap_layer varaible to heatmap_layer attribute and add in locations
#add heatmap layer
#call figure to plot data


In [ ]:
#heatmap of temperature
#get lats and lngs
locations = city_data_df[["Lat", "Lng"]]
#get max temp
max_temp = city_data_df["Max Temp"]
#assign the figure variable
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
#add heatmap layer
fig.add_layer(heat_layer)
#call fig to plot
fig

In [ ]:
#heatmap of percent humidity 
#get lats and lngs
locations = city_data_df[["Lat", "Lng"]]
#get max temp
humidity = city_data_df["Humidity"]
#assign the figure variable
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
#add heatmap layer
fig.add_layer(heat_layer)
#call fig to plot
fig

In [ ]:
#get lats and lngs
locations = city_data_df[["Lat", "Lng"]]
#get max temp
clouds = city_data_df["Cloudiness"]
#assign the figure variable
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
#add heatmap layer
fig.add_layer(heat_layer)
#call fig to plot
fig

In [ ]:
#get lats and lngs
locations = city_data_df[["Lat", "Lng"]]
#get max temp
wind = city_data_df["Wind Speed"]
#assign the figure variable
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
#add heatmap layer
fig.add_layer(heat_layer)
#call fig to plot
fig

In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
#filter data based on customer input
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head(10)

In [ ]:
preferred_cities_df.count()

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
#hotel search parameters
params = {"radius": 5000, "type":"lodging", "key":g_key}
#iterate thru the dataframe
for index, row in hotel_df.iterrows():
    #get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    #add lat and lng to location key for params dict
    params["location"] = f"{lat},{lng}"
    #use search term lodging and lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make request and get the json data from search
    hotels = requests.get(base_url, params=params).json()
    #grab first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# add a heatmap of temp for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig